In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
import pandas as pd

wine = pd.read_csv('../data/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [20]:
data = wine.iloc[:, :3].to_numpy()
target = wine.iloc[:, 3].to_numpy()

In [21]:
data

array([[ 9.4 ,  1.9 ,  3.51],
       [ 9.8 ,  2.6 ,  3.2 ],
       [ 9.8 ,  2.3 ,  3.26],
       ...,
       [ 9.4 ,  1.2 ,  2.99],
       [12.8 ,  1.1 ,  3.34],
       [11.8 ,  0.8 ,  3.26]])

In [22]:
target

array([0., 0., 0., ..., 1., 1., 1.])

In [23]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data,
    target, 
    test_size=0.2,
    random_state=42,
)

In [24]:
# RandomForest Model

import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42, n_estimators=100)

scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8903229806766861


In [25]:
# 주요 Feature 확인

rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23183515 0.50059756 0.26756729]


In [26]:
# 부트스트랩 결정시 남는 샘플(oob out of back)도 특성으로 구분할 수 있다.

rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8945545507023283


---
### Extra Tree(엑스트라 트리)
- 기본적으로 100개의 트리를 사용
- 노드 분할시 특성(Feature)의 제곱근을 갯수로 사용
- 특성 선택을 랜덤하게 선택
- 특성의 선택을 랜덤하게 하므로, 속도는 랜덤포레스트보다 빠르다.

In [27]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
# 주요 Feature 확인
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23183515 0.50059756 0.26756729]


---
### Gradient Boosting(그래디언트 부스팅)
- 가장 유명한 알고리즘중 하나이다.
- 경사하강법처럼 손실함수를 사용.
- 손실함수를 보고 트리를 추가하여 최적의 값을 도출하는 방법
- Decisino Tree Regressor를 사용하여 손실함수를 계산한고 이를 계속 낮추기 위해 트리를 추가하는 구조
- 경사를 이동하면서 경사의 이동거리를 제어하는 learning-rate(기본 : 0.1)을 사용한다.  
- max depth를 3으로 제어하여 깊이가 낮으므로, 과대적합이 방지된다.
- 단점은 손실함수를 보고 트리를 추가하면서 진행하는 모델이므로 병렬처리(n_jobs)를 할수 없다.

In [30]:
# 기본값으로 모델 구성
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [31]:
# 기본값 확인

gb = GradientBoostingClassifier(random_state=42, n_estimators=100, learning_rate=0.1)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [36]:
# 기본값 변경
gb = GradientBoostingClassifier(random_state=42, n_estimators=500, learning_rate=0.02)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8872908264055726 0.8720428296438882


In [37]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.12190897 0.7416382  0.13645283]


---
### 히스토그램 기반 그래디언트 부스팅(Histogram gradient boosting)
- 훈련데이터를 256개의 구간으로 나누어서 훈련시키는 방법
- 특성의 범위가 제한되어 빠른 속도를 제공함.
- 제한된 구간이므로 과대적합을 방지.
- 아직은 실험단계인 모델.

In [38]:
# 모델 새성

from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


---
# * 번외 기능 *
### Permutation Importance(치환 중요도)
- feature별 sample을 섞어서 계산을 한 후에 원래 sample들과의 차이를 계산하여 차이가 많이 나는 featrue가 중요하다는 판단을 한다. 
- 즉, 어느 Feature가 중요한지 파악하는 방법.
- 어떤 모델에도 사용 가능하며, 특성을 파악하는 주요 기준으로 사용된다. 권장사항
- Dataset을 구성할 때 필수적으로 사용되는 모델이며, 수집한 데이터의 중요도 파악에 많이 사용되며 영향이 없는 Feature는 속도만 축내는 꼴이다..


In [41]:
# Train경우 확인

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [42]:
print(result)

{'importances_mean': array([0.08876275, 0.23438522, 0.08027708]), 'importances_std': array([0.00382333, 0.00401363, 0.00477012]), 'importances': array([[0.08793535, 0.08350972, 0.08908986, 0.08312488, 0.09274581,
        0.08755051, 0.08601116, 0.09601693, 0.09082163, 0.09082163],
       [0.22782374, 0.23590533, 0.23936887, 0.23436598, 0.23725226,
        0.23436598, 0.23359631, 0.23398114, 0.23994612, 0.22724649],
       [0.08581874, 0.08601116, 0.08062344, 0.07504329, 0.08427939,
        0.07792957, 0.07234943, 0.07465846, 0.08139311, 0.08466423]])}


> sugar의 경우 Feature를 무작위로 섞으면 23%만큼 정확도가 떨어진다.  


---
### XGBoost
- kaggle에서 많이 사용됨.

In [43]:
from xgboost import XGBClassifier

In [46]:
xgb = XGBClassifier(
            tree_method = 'hist',
            random_state = 42, 
            eval_metric = 'logloss',
)

scores =  cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686
